In [1]:
import pandas as pd

In [2]:
# !pip install psycopg2
import psycopg2

# Connect to your postgres DB
conn = psycopg2.connect(database="postgres", host="is-postgres-au-vic.postgres.database.azure.com", user="ds@is-postgres-au-vic", password='P":VQ:7yD?ha!r7"yWsmT15n7-RbT')

# Open a cursor to perform database operations
cur = conn.cursor()

In [17]:
query = """

with cohort as (
select stu.stu_uuid
from ds.mv_students as stu, (
select tnt.tnt_uuid 
from (
    select tnt_uuid, count(*) as cnt 
    from ds.mv_external_results as ext
    where ext.external_result_type_name='NAPLAN' and ext.assessment_component='Numeracy'
    group by tnt_uuid
    order by count(*) desc
    limit 1
) as tnt) as tnt_id
where tnt_id.tnt_uuid=stu.tnt_uuid and stu.current_year_level=10
)


select * 
from ds.mv_assessment_results as res, cohort, ds.mv_timetabled_classes as tt, ds.mv_learning_domain as ldom
where res.stu_uuid=cohort.stu_uuid and res.ttclass_uuid=tt.ttclass_uuid and tt.ldom_uuid=ldom.ldom_uuid and ldom.learning_domain='Mathematics'

"""

In [20]:
cur.execute(query)

# Retrieve query results
records = cur.fetchall()

In [35]:
internals = pd.DataFrame(records)
colnames = [desc[0] for desc in cur.description]
internals.columns = colnames
internals.head()

,tnt_uuid,assessment_uuid,ttsess_uuid,stu_uuid,ttclass_uuid,status,name,activate,assessment_result_uuid,result_numeric,...,session_start,session_end,status,title,school_year,tnt_uuid,ldom_uuid,learning_domain,learning_domain_code,learning_domain_enum
0,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e3e986-bc32-0a22-9085-0d6088b6d309,67045c9c-21be-879c-1774-9b6a2d22a73b,3ff4e152-fe23-90f1-2dc5-795379649da8,443eb7a2-e3f6-ca47-669c-e72638cea15a,ASSIGNED,Mathematics Semester Result G08MT4 K,2020-11-26 14:00:00+00:00,c512cee0-6765-6cd8-64d8-63953fcec12c,0.960000,...,2020-06-15,2020-12-09,active,Semester 2,2020,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,None
1,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e35d50-9b2f-f8ae-6555-58f0949d9b66,67046440-439d-5c20-d62f-8ddc655052cf,3ff4e152-fe23-90f1-2dc5-795379649da8,443d72a3-8eaa-c49e-bdf5-58efb72b9e32,ASSIGNED,Mathematics Semester Result G07MT1 K,2019-06-13 14:00:00+00:00,c508f21c-bdb6-5f80-3b7a-bacfdef687c6,0.970000,...,2019-01-29,2019-06-14,active,Semester 1,2019,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,None
2,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e44eae-1534-f120-29e1-5d0abff52050,6704c9c1-2c77-0533-e04a-e73fa620c87b,3ff4e152-fe23-90f1-2dc5-795379649da8,443f3130-1ca9-65f0-852c-ab39aeb5be6f,ASSIGNED,Mathematics Semester One Grade G08MT4 K,2020-06-11 14:00:00+00:00,c508f942-d586-1a52-6d89-c228eaf745df,0.920000,...,2020-01-31,2020-06-12,active,Semester 1,2020,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,None
3,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e49d22-9205-e2b7-afdc-312fd9564810,670498db-d726-8504-5ac1-d28ef68b5bcb,3ff4e152-fe23-90f1-2dc5-795379649da8,443e4335-7376-0acd-2d95-20751949a458,ASSIGNED,Mathematics Semester result G07MT1 K,2019-12-01 14:00:00+00:00,c50b8f96-1903-9cfa-4505-f01ca31d6f43,0.950000,...,2019-06-17,2019-12-11,active,Semester 2,2019,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,None
4,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e46ea5-e4c8-a3f1-a72d-324c504d8fb3,67044b53-5747-9015-5532-332cc438841a,3ff4e152-fe23-90f1-2dc5-795379649da8,443fa9f5-a241-7aea-31b4-332332ae5592,ASSIGNED,Mathematics A Semester result G09MA4 K,2021-11-24 14:00:00+00:00,c51f0189-2fa0-f43e-48c3-3d5e3c155b60,0.870000,...,2021-07-13,2021-12-10,active,Semester 2,2021,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,None


In [3]:
query = """

with cohort as (
select stu.stu_uuid
from ds.mv_students as stu, (
select tnt.tnt_uuid 
from (
    select tnt_uuid, count(*) as cnt 
    from ds.mv_external_results as ext
    where ext.external_result_type_name='NAPLAN' and ext.assessment_component='Numeracy'
    group by tnt_uuid
    order by count(*) desc
    limit 1
) as tnt) as tnt_id
where tnt_id.tnt_uuid=stu.tnt_uuid and stu.current_year_level=10
)


select * 
from ds.mv_external_results as ext, cohort
where ext.stu_uuid=cohort.stu_uuid and ext.external_result_type_name='NAPLAN' and ext.assessment_component='Numeracy'

"""

cur.execute(query)

# Retrieve query results
ext_records = cur.fetchall()

externals = pd.DataFrame(ext_records)
colnames_ext = [desc[0] for desc in cur.description]
externals.columns = colnames_ext
externals.head()

,tnt_uuid,extr_uuid,lextest_uuid,stu_uuid,type,description,date_taken,test_year,year_level,score,...,percentile,assessment_component,external_result_type_name,external_result_type_description,external_result_type_code,external_result_type_max_score,external_result_type_min_score,group_label,name,stu_uuid
0,6d27c189-cc31-4970-a6b2-49019ce66c3f,b0bbafc4-1ea4-da52-1835-843c41a818d7,349c7e04-e97c-666b-b043-53375920c0ec,3ff4128c-5d61-79fb-b656-40ddfe51b4b4,NAPLAN,NAPLAN Numeracy,2021-05-30,2021,9,555.000000000,...,0.16020236087689713000,Numeracy,NAPLAN,None,None,1000,0,band,NAPLAN 2021: Year 9,3ff4128c-5d61-79fb-b656-40ddfe51b4b4
1,6d27c189-cc31-4970-a6b2-49019ce66c3f,b0bbcdaa-232e-c3ae-d475-3e1eb5f7030a,349c7e04-e97c-666b-b043-53375920c0ec,3ff40118-8025-e343-8558-1c346fc0145a,NAPLAN,NAPLAN Numeracy,2021-05-30,2021,9,796.000000000,...,0.9730185497470488,Numeracy,NAPLAN,None,None,1000,0,band,NAPLAN 2021: Year 9,3ff40118-8025-e343-8558-1c346fc0145a
2,6d27c189-cc31-4970-a6b2-49019ce66c3f,b0bb5ddd-e141-738e-4271-8b0603f025a0,349c7e04-e97c-666b-b043-53375920c0ec,3ff40fdb-4210-4f87-ade0-6becaaa3ecd0,NAPLAN,NAPLAN Numeracy,2021-05-30,2021,9,586.000000000,...,0.2833052276559865,Numeracy,NAPLAN,None,None,1000,0,band,NAPLAN 2021: Year 9,3ff40fdb-4210-4f87-ade0-6becaaa3ecd0
3,6d27c189-cc31-4970-a6b2-49019ce66c3f,b0bbe13f-601e-0be6-5db7-e9b3510789a4,349c7e04-e97c-666b-b043-53375920c0ec,3ff4e823-be46-323f-a107-c988ef8a2319,NAPLAN,NAPLAN Numeracy,2021-05-30,2021,9,632.000000000,...,0.5497470489038786,Numeracy,NAPLAN,None,None,1000,0,band,NAPLAN 2021: Year 9,3ff4e823-be46-323f-a107-c988ef8a2319
4,6d27c189-cc31-4970-a6b2-49019ce66c3f,b0bb220f-22ff-8bee-d8a4-5f3054ccfb8b,349c7e04-e97c-666b-b043-53375920c0ec,3ff4362f-e3e2-228d-2b1e-4382f3c321e4,NAPLAN,NAPLAN Numeracy,2021-05-30,2021,9,677.000000000,...,0.7419898819561551,Numeracy,NAPLAN,None,None,1000,0,band,NAPLAN 2021: Year 9,3ff4362f-e3e2-228d-2b1e-4382f3c321e4


In [43]:
internals.to_csv('internals.csv', header=True, index=False)
externals.to_csv('externals.csv', header=True, index=False)

In [18]:
conn = psycopg2.connect(database="postgres", host="is-postgres-au-vic.postgres.database.azure.com", user="ds@is-postgres-au-vic", password='P":VQ:7yD?ha!r7"yWsmT15n7-RbT')
cur = conn.cursor()

In [22]:
query3 = """

with cohort as (
select stu.stu_uuid
from ds.mv_students as stu, (
select tnt.tnt_uuid 
from (
    select tnt_uuid, count(*) as cnt 
    from ds.mv_external_results as ext
    where ext.external_result_type_name='NAPLAN' and ext.assessment_component='Numeracy'
    group by tnt_uuid
    order by count(*) desc
    limit 1
) as tnt) as tnt_id
where tnt_id.tnt_uuid=stu.tnt_uuid and stu.current_year_level=10
)

select lin.stu_uuid, ext.test_year, qst.*, lin.result_numeric, lin.result_text, lin.response, lin.is_correct, lin.weight, lin.attempted, lin.presented
from cohort, ds.mv_external_result_question as qst, ds.mv_external_result_line as lin, ds.mv_external_results as ext
where lin.lertq_uuid=qst.lerq_uuid and ext.extr_uuid=lin.extr_uuid and
ext.stu_uuid=cohort.stu_uuid and ext.external_result_type_name='NAPLAN' and ext.assessment_component='Numeracy'

"""

cur.execute(query3)

# Retrieve query results
ext_details = cur.fetchall()

ext_details_df = pd.DataFrame(ext_details)
colnames_ext_details = [desc[0] for desc in cur.description]
ext_details_df.columns = colnames_ext_details
ext_details_df.head()

,stu_uuid,test_year,lerq_uuid,ext_id,question,correct_response,lldom_uuid,description,sort_order,max_score,...,lcarea_uuids,lccriterion_uuids,type,result_numeric,result_text,response,is_correct,weight,attempted,presented
0,3ff48b2a-3132-5a05-74f9-8a6ab2f5e24a,2017,a3b1daed-e209-49b1-01cd-cd3c78736404,2017-05|NUMERACY-08,None,A,3490b4d6-2ae6-25cb-088a-9962258b225d,None,None,None,...,None,None,Multiple choice,None,None,None,True,None,True,True
1,3ff48b2a-3132-5a05-74f9-8a6ab2f5e24a,2017,a3b155e8-9e96-b10d-8e2f-902e55545d70,2017-05|NUMERACY-01,None,C,3490b4d6-2ae6-25cb-088a-9962258b225d,None,None,None,...,None,None,Multiple choice,None,None,None,True,None,True,True
2,3ff48b2a-3132-5a05-74f9-8a6ab2f5e24a,2017,a3b13c69-575f-d7c1-a9f3-50b76234fc97,2017-05|NUMERACY-02,None,B,3490b4d6-2ae6-25cb-088a-9962258b225d,None,None,None,...,None,None,Multiple choice,None,None,None,True,None,True,True
3,3ff48b2a-3132-5a05-74f9-8a6ab2f5e24a,2017,a3b1944c-d19d-59ba-2a6a-d339a39c8572,2017-05|NUMERACY-03,None,B,3490b4d6-2ae6-25cb-088a-9962258b225d,None,None,None,...,None,None,Multiple choice,None,None,None,True,None,True,True
4,3ff48b2a-3132-5a05-74f9-8a6ab2f5e24a,2017,a3b1c5ac-bb62-eee0-56a5-3252fd558485,2017-05|NUMERACY-04,None,B,3490b4d6-2ae6-25cb-088a-9962258b225d,None,None,None,...,None,None,Multiple choice,None,None,None,True,None,True,True


In [23]:
ext_details_df.to_csv('ext_details_df.csv', header=True, index=False)